## Análisis de Sentimientos de críticas de películas

Junto con Keras, viene un ejemplo imdb_lstm.py. Este ejercicio esta prácticamente basado en él.

Es un gran ejemplo del uso de las RNNs.  El conjunto de datos que se utilizará consta de críticas de películas generadas por usuarios, y una classificación indicando si le gustó, o no, basado en su rating asociado. 

Hay más información de este conjunto de datos en:

https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

Como la comprensión del lenguaje escrito requiere "llevar cuenta" de todas las palabras en una oración, necesitamos una RNN para mantener una "memoria" de las palabras que pasaron antes, conforme va "leyendo" oraciones a lo largo del tiempo. 

En particular, se usarán unidades LSTM (Long Short-Term Memory) porque no es deseable "olvidar" palabras demasiado rápido...las palabras al inicio de una oración pueden afectar el significado de la misma grandemente.

Empezamos por la importación de lo que se requiere:

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

Ahora importar los datos para entrenamiento y prueba.  Para que sea más manejable, se especifica que se quieren solamente las 20,000 palabras más populares en el conjunto de datos. Por algún motivo, este conjunto tiene una relación de 50% entreno y 50% prueba. 

In [ ]:
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

A ver cómo son los datos, el primer elemento de entrenamiento debe ser una crítica de una película:

In [ ]:
X_train[0]

Esto no parece una crítica de una película!!!!  Resulta que la gente que preparó los datos ya hicieron algo de preparación previa de los datos.  Estos números coinciden con el índice correspondiente a cada palabra de la crítica.  En realidad las palabras en sí, no son de interés...el modelo requiere números no palabras. 

Lo triste es que no será posible leer las críticas...siquiera para tener una idea de si funciona el análiisis, o no.

Y, ¿cómo son las etiquetas (metas)?

In [ ]:
y_train[0]

Son simplemente 0 ó 1, que indica sí al que escribió la crítica le gustó, o no, la película.

Para resumir, para el entrenamiento se tiene un conjunto de críticas de películas que han sido convertidas a vectores de palabras representadas por enteros, y una clasificación de sentimiento binaria.

Las RNNs pueden "explotar""
 (se habló de esto en clase) muy rápidamente.  Para que no se sobrecarguen las PCs que se podrían usar, se limitarán las críticas a las primeras 80 palabras:

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen = 80)
X_test = sequence.pad_sequences(X_test, maxlen = 80)

## Configuración del modelo

Considerando lo complejo que es una RNN LSTM (debajo del "capó"), es increíble lo simple que resulta esta parte utilizando Keras.

Se empieza con una capa de incrustación "embedded".  La función de esta es convertir los datos de entrada a vectores densos de un tamaño fijo que son más adecuados para una red neuronal.  Esto es típico cuando se manejan indices de datos basados en texto.  El 20,000 representa el tamaño del vocabulario (límite impuesto para este ejercicio, pero puede variarse dependiendo de la capacidad de cómputo que se tiene) y 128 es la dimensión de 128 unidades de salida.

Luego se coloca una capa LSTM y ya!  Se especifica 128 para igualar la salida de la capa de incrustación, y se utiliza la opción de "botar" unidades para evitar el sobre ajuste, que es una característica muy común de las RNN.

Finalmente se debe reducir todo a una unidad de salida con una función de activación sigmoidal para seleccionar la clasificación binaria de sentimiento de 0 ó 1.

In [ ]:
modelo = Sequential()
modelo.add(Embedding(20000, 128))
modelo.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
modelo.add(Dense(1, activation='sigmoid'))

Ya que este es un problema de clasificación binaria, la mejor función de pérdida es la "binary_crossentropy".  También se utiliza el optimizador "Adam" que es uno de los mejores.  Siempre es de recordar que si es necesario afinar más el modelo, se pueden probar otras.

In [ ]:
modelo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Entrenamiento

Las RNNs, al igual que las CNNs son bastante pesadas en el uso de recursos.  Para poder utilizar una PC normal, el mantener el tamaño de las tandas pequeño, es clave.  En un mundo más profesional, se estaría sacando ventaja de GPUs instaladas en un cluster de computadoras para tener un mejor rendimiento.

## Advertencia

Esto puede tardarse bastante tiempo, aún en una PC potente!  A menos que se quiera ver todo el proceso y esperar una o más horas, no ejecute las siguientes celdas de código.

Esta siguiente celda es solo para quienes estén utilizando tensorflow-gpu con Windows.  Sirve para evitar problemas de asignación de memoria.

In [ ]:
import tensorflow as tf
from tensorflow.keras.backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # hacer que la memoria usada del GPU crezca dinamicamente 
sess = tf.Session(config = config)
set_session(sess)  # hacer que esta sesión de TensorFlow sea la sesión default de Keras

Ahora sí, a iniciar el entrenamiento.  Aún con una GPU se tardará bastante tiempo!

In [ ]:
modelo.fit(X_train, y_train,
          batch_size = 32,
          epochs = 15,
          verbose = 2,
          validation_data = (X_test, y_test))

OK, ahora a evaluar la exactitud del modelo:

In [ ]:
perdida, exactitud = modelo.evaluate(X_test, y_test,
                            batch_size = 32,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)

Cerca del 80%, no está mal considerando que se limitó el ejercicio a las primeras 80 palabras de cada crítica.

Por otro lado - hay que pensar en lo que se hizo en este ejercicio!  Una red neuronal que puede leer críticas y deducir si al autor le gustó la película o no, basado en el texto.  Y el modelo toma en cuenta el contexto y la posición de cada palabra.

Lo mejor de todo es que armar el modelo solamente requirió de algunas líneas de código!  Es increíble lo que se puede hacer con Keras!!!

In [ ]:
modelo.save("Anal_Sentimiento.h5")

### Ver algo de las críticas

Del enlace provisto arriba, se obtuvo el siguiente código que permite ver el texto del primero de los comentarios.  No sería dificil verlos todos si se quisiera.

In [ ]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3
# Retrieve the training sequences.
(X_train, _), _ = tf.keras.datasets.imdb.load_data(
    start_char=start_char, oov_char=oov_char, index_from=index_from
)
# Retrieve the word index file mapping words to indices
word_index = tf.keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in X_train[0])

In [ ]:
decoded_sequence